# Intro to Data Structures and Algorithms 

[course link](https://learn.udacity.com/courses/ud513)

## Lesson 7. Case Studies in Algorithms

### Shortest Path Problem

### Dijkstra's Algorithm 

### Knapsack Problem 

### A Faster Algorithm

### Dynamic Programming

### Travelling Salesman Problem

### Exact and Approximate Algorithms 